##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Penguin 模板为您的数据创建 TFX 流水线

---


注：我们建议在 Google Cloud Vertex AI Workbench 上运行本教程。[前往 Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench)。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_template"> <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_template.ipynb"> <img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_template.ipynb"> <img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

## 简介

本文档将提供使用随 TFX Python 软件包提供的 *penguin 模板*为您自己的数据集创建 TensorFlow Extended (TFX) 流水线的说明。创建的流水线最初将使用 [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) 数据集，但我们将为您的数据集转换流水线。


### 前提条件

- Linux / MacOS
- Python 3.6-3.8
- Jupyter Notebook


## 第 1 步：将预定义的模板复制到项目目录

在此步骤中，我们将从 TFX 中的 *penguin 模板*复制文件以创建工作流水线项目目录和文件。您可以将此视为您的 TFX 流水线项目的基架。

### 更新 Pip

如果在 Colab 中运行，我们应确保拥有最新版本的 Pip。本地系统当然可以单独更新。

注：如果您在 Vertex AI Workbench 中运行，更新可能也是一个好主意。

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### 安装所需的软件包

首先，安装 TFX 和 TensorFlow Model Analysis (TFMA)。


In [ ]:
!pip install -U tfx tensorflow-model-analysis

我们来检查一下 TFX 的版本。

In [ ]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tfx

print('TF version: {}'.format(tf.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFX version: {}'.format(tfx.__version__))

我们已完成创建流水线的准备工作。

将 `PROJECT_DIR` 设置为适用于您的环境的目标。默认值为 `~/imported/${PIPELINE_NAME}`，适用于 [Google Cloud AI Platform Notebook](https://console.cloud.google.com/ai-platform/notebooks/) 环境。

您可以通过更改下面的 `PIPELINE_NAME` 为流水线使用其他名称。这也将成为存放文件的项目目录的名称。


In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Set this project directory to your new tfx pipeline project.
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "imported", PIPELINE_NAME)

### 复制模板文件

TFX 包含带有 TFX python 软件包的 `penguin` 模板。`penguin` 模板包含许多为您的数据集创建流水线的说明，本教程的目的正是讲解这一内容。

`tfx template copy` CLI 命令会将预定义的模板文件复制到您的项目目录中。

In [ ]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=penguin

将此笔记本中的工作目录上下文更改为项目目录。

In [ ]:
%cd {PROJECT_DIR}

> 注：如果您使用的是 JupyterLab 或 Google Cloud AI Platform Notebook，则创建项目目录后，不要忘记点击进入项目目录来更改左侧 `File Browser` 中的目录。

### 浏览复制的源文件

TFX 模板提供用于构建流水线的基本基架文件，包括 Python 源代码和样本数据。`penguin` 模板使用与[企鹅示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)相同的 *Palmer Penguins* 数据集和机器学习模型。

以下是每个 Python 文件的简要说明。

- `pipeline` - 此目录包含流水线的定义
    - `constants.py` - 定义流水线运行程序的通用常量
    - `pipeline.py` - 定义 TFX 组件和流水线
- `models` - 此目录包含机器学习模型定义
    - `features.py`、`features_test.py` - 定义模型的特征
    - `preprocessing.py`、`preprocessing_test.py` - 定义数据的预处理例程
    - `constants.py` - 定义模型的常量
    - `model.py`、`model_test.py` - 使用 TensorFlow 等机器学习框架定义机器学习模型
- `local_runner.py` - 为使用本地编排引擎的本地环境定义运行引擎
- `kubeflow_runner.py` - 为 Kubeflow Pipelines 编排引擎定义运行程序


默认情况下，模板仅包含标准 TFX 组件。如果您需要一些自定义操作，可以为您的流水线创建自定义组件。请参阅 [TFX 自定义组件指南](https://tensorflow.google.cn/tfx/guide/understanding_custom_components)以了解详情。

#### 单元测试文件

您可能注意到某些文件的名称中包含 `_test.py`。这些是流水线的单元测试，建议您在实现自己的流水线时添加更多单元测试。您可以通过提供带有 `-m` 标记的测试文件的模块名称来运行单元测试。通常，您可以通过删除 `.py` 扩展名并将 `/` 替换为 `.` 来获得模块名称。例如：

In [ ]:
import sys
!{sys.executable} -m models.features_test

### 在本地环境中创建 TFX 流水线

TFX 支持使用多种编排引擎来运行流水线。我们将使用本地编排引擎。本地编排引擎运行不需要任何后续依赖项，并且因其运行于本地环境而不依赖于远程计算集群，非常适于开发和调试。

我们将使用 `local_runner.py` 通过本地编排器运行您的流水线。您必须在运行之前先创建流水线。您可以使用 `pipeline create` 命令创建流水线。


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

`pipeline create` 命令会注册您在 `local_runner.py` 中定义的流水线，而非实际运行它。

在以下步骤中，您将使用 `run create` 命令运行创建的流水线。


## 第 2 步：将您的数据提取到流水线中

初始流水线会提取模板中包含的企鹅数据集。您需要将数据置入流水线中，大多数 TFX 流水线都以 ExampleGen 组件开头。

### 选择 ExampleGen

您的数据可以存储在您的流水线可访问的任何位置，在本地或分布式文件系统或可查询系统上均可。TFX 提供了多种 [`ExampleGen` 组件](https://tensorflow.google.cn/tfx/guide/examplegen)可用于将您的数据汇入 TFX 流水线。您可以从以下样本生成组件中选择一种。

- CsvExampleGen：读取目录中的 CSV 文件。[企鹅示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)和[芝加哥出租车示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)中使用了此组件。
- ImportExampleGen：接受具有 tf.Example 数据格式的 TFRecord 文件。[MNIST 示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/mnist)中使用了此组件。
- FileBasedExampleGen：用于 [Avro](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/avro_executor.py) 或 [Parquet](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/parquet_executor.py) 格式。
- [BigQueryExampleGen](https://tensorflow.google.cn/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen)：直接读取 Google Cloud BigQuery 中的数据。[芝加哥出租车示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)中使用了此组件。

您还可以创建自己的 ExampleGen，例如，tfx 包含[使用 Presto 作为数据源的自定义 ExecampleGen](https://github.com/tensorflow/tfx/tree/master/tfx/examples/custom_components/presto_example_gen)。有关如何使用和开发自定义执行器的详细信息，请参阅[指南](https://tensorflow.google.cn/tfx/guide/examplegen#custom_examplegen)。

决定使用哪种 ExampleGen 后，您将需要修改流水线定义以使用您的数据。

1. 修改 `local_runner.py` 中的 `DATA_PATH` 并将其设置为您的文件的位置。

- 如果您具有本地环境中的文件，请指定路径。这是开发或调试流水线时的最佳选择。
- 如果文件存储在 GCS 中，您可以使用以 `gs://{bucket_name}/...` 开头的路径。请确保您可以从终端访问 GCS，例如使用 [`gsutil`](https://cloud.google.com/storage/docs/gsutil)。如果需要，请遵循 [Google Cloud 中的授权指南](https://cloud.google.com/sdk/docs/authorizing)。
- 如果您想使用诸如 BigQueryExampleGen 的基于查询的 ExampleGen，则您需要 Query 语句以从数据源中选择数据。要使用 Google Cloud BigQuery 作为数据源，您还需要进行一些额外设置。
    - 在 `pipeline/configs.py` 中：
        - 将 `GOOGLE_CLOUD_PROJECT` 和 `GCS_BUCKET_NAME` 更改为您的 GCP 项目和存储分区名称。存储分区应在我们运行流水线之前就已存在。
        - 取消注释 `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` 变量。
        - 取消注释 `BIG_QUERY_QUERY` 变量并将其设置为**您的查询语句**。
    - 在 `local_runner.py` 中：
        - 在 `pipeline.create_pipeline()` 中注释掉 `data_path` 参数并取消注释 `query` 参数。
    - 在 `pipeline/pipeline.py` 中：
        - 在 `create_pipeline()` 中注释掉 `data_path` 参数并取消注释 `query` 参数。
        - 使用 [BigQueryExampleGen](https://tensorflow.google.cn/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen) 而非 CsvExampleGen。

1. 在 `pipeline/pipeline.py` 中将现有的 CsvExampleGen 替换为您的 ExampleGen 类。每个 ExampleGen 类都具有不同的签名。请参阅 [ExampleGen 组件指南](https://tensorflow.google.cn/tfx/guide/examplegen)，了解更多详细信息。不要忘记在 `pipeline/pipeline.py` 中使用 `import` 语句导入所需的模块。

初始流水线包含四个组件：`ExampleGen`、`StatisticsGen`、`SchemaGen` 和 `ExampleValidator`。我们不需要对 `StatisticsGen`、`SchemaGen` 和 `ExampleValidator` 进行任何更改。我们来首次运行流水线。

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

如果流水线成功运行，您应该看到“Component ExampleValidator is finished.”。

### 检查流水线的输出

TFX 流水线会生成两种输出：工件和[元数据 DB(MLMD)](https://tensorflow.google.cn/tfx/guide/mlmd)，后者包含工件和流水线执行的元数据。输出的位置在 `local_runner.py` 中定义。默认情况下，工件存储在 `tfx_pipeline_output` 目录下，元数据以 sqlite 数据库形式存储在 `tfx_metadata` 目录下。

您可以使用 MLMD API 检查这些输出。首先，我们将定义一些效用程序函数来搜索刚刚生成的输出工件。

In [ ]:
import tensorflow as tf
import tfx
from ml_metadata import errors
from ml_metadata.proto import metadata_store_pb2
from tfx.types import artifact_utils

# TODO(b/171447278): Move these functions into TFX library.

def get_latest_executions(store, pipeline_name, component_id = None):
  """Fetch all pipeline runs."""
  if component_id is None:  # Find entire pipeline runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('run')
        if c.properties['pipeline_name'].string_value == pipeline_name
    ]
  else:  # Find specific component runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('component_run')
        if c.properties['pipeline_name'].string_value == pipeline_name and
           c.properties['component_id'].string_value == component_id
    ]
  if not run_contexts:
    return []
  # Pick the latest run context.
  latest_context = max(run_contexts,
                       key=lambda c: c.last_update_time_since_epoch)
  return store.get_executions_by_context(latest_context.id)

def get_latest_artifacts(store, pipeline_name, component_id = None):
  """Fetch all artifacts from latest pipeline execution."""
  executions = get_latest_executions(store, pipeline_name, component_id)

  # Fetch all artifacts produced from the given executions.
  execution_ids = [e.id for e in executions]
  events = store.get_events_by_execution_ids(execution_ids)
  artifact_ids = [
      event.artifact_id for event in events
      if event.type == metadata_store_pb2.Event.OUTPUT
  ]
  return store.get_artifacts_by_id(artifact_ids)

def find_latest_artifacts_by_type(store, artifacts, artifact_type):
  """Get the latest artifacts of a specified type."""
  # Get type information from MLMD
  try:
    artifact_type = store.get_artifact_type(artifact_type)
  except errors.NotFoundError:
    return []
  # Filter artifacts with type.
  filtered_artifacts = [aritfact for aritfact in artifacts
                        if aritfact.type_id == artifact_type.id]
  # Convert MLMD artifact data into TFX Artifact instances.
  return [artifact_utils.deserialize_artifact(artifact_type, artifact)
      for artifact in filtered_artifacts]


from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

import pprint

from tfx.orchestration import metadata
from tfx.types import artifact_utils
from tfx.types import standard_artifacts

def preview_examples(artifacts):
  """Preview a few records from Examples artifacts."""
  pp = pprint.PrettyPrinter()
  for artifact in artifacts:
    print("==== Examples artifact:{}({})".format(artifact.name, artifact.uri))
    for split in artifact_utils.decode_split_names(artifact.split_names):
      print("==== Reading from split:{}".format(split))
      split_uri = artifact_utils.get_split_uri([artifact], split)

      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = [os.path.join(split_uri, name)
                            for name in os.listdir(split_uri)]
      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames,
                                        compression_type="GZIP")
      # Iterate over the first 2 records and decode them.
      for tfrecord in dataset.take(2):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        pp.pprint(example)

import local_runner

metadata_connection_config = metadata.sqlite_metadata_connection_config(
              local_runner.METADATA_PATH)

现在，我们可以从 MLMD 读取输出工件的元数据。

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous pipeline run.
    artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
    # Find artifacts of Examples type.
    examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)
    # Find artifacts generated from StatisticsGen.
    stats_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleStatistics.TYPE_NAME)
    # Find artifacts generated from SchemaGen.
    schema_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Schema.TYPE_NAME)
    # Find artifacts generated from ExampleValidator.
    anomalies_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleAnomalies.TYPE_NAME)

现在，我们可以检查每个组件的输出。[Tensorflow Data Validation (TFDV)](https://tensorflow.google.cn/tfx/data_validation/get_started) 在 `StatisticsGen`、`SchemaGen` 和 `ExampleValidator` 中使用，可以使用 TFDV 呈现这些组件的输出。

在本教程中，我们将使用 TFX 中的呈现辅助方法，这些方法在内部使用 TFDV 来显示呈现效果。请参阅 [TFX 组件教程](https://tensorflow.google.cn/tfx/tutorials/tfx/components_keras)以详细了解每个组件。

#### 检查 ExampleGen 的输出

让我们检查一下 ExampleGen 的输出。看一下每个拆分的前两个样本：

In [ ]:
preview_examples(examples_artifacts)

默认情况下，TFX ExampleGen 会将样本划分到 *train* 和 *eval* 两个拆分中，但您可以[调整拆分配置](https://tensorflow.google.cn/tfx/guide/examplegen#span_version_and_split)。

#### 检查 StatisticsGen 的输出


In [ ]:
visualize_artifacts(stats_artifacts)

这些统计信息被提供给 SchemaGen 以自动构造数据模式。

#### 检查 SchemaGen 的输出


In [ ]:
visualize_artifacts(schema_artifacts)

此模式基于 StatisticsGen 的输出自动推断。我们将在本教程中使用此生成的模式，但您也可以[修改和自定义模式](https://tensorflow.google.cn/tfx/guide/statsgen#creating_a_curated_schema)。

#### 检查 ExampleValidator 的输出


In [ ]:
visualize_artifacts(anomalies_artifacts)

如果发现任何异常，您可以检查数据以确认所有样本是否均符合假设。其他组件（如 StatistcsGen）的输出可能很有用。发现的异常不会阻止流水线执行。

您可以从 `SchemaGen` 的输出查看可用特征。如果您的特征可以直接在 `Trainer` 中用于构造机器学习模型，则可以跳过下一步并前往第 4 步。否则，您可以在下一步中进行一些特征工程工作。当需要诸如计算平均值的全通运算时（尤其是当您需要缩放时），需要 `Transform` 组件。

## 第 3 步（可选）：使用 Transform 组件进行特征工程

在此步骤中，您将定义流水线中的 `Transform` 组件将使用的各种特征工程作业。有关详情，请参阅 [Transform 组件指南](https://tensorflow.google.cn/tfx/guide/transform)。

只有当您的训练代码需要在 ExampleGen 的输出中不可用的附加特征时，才需要此步骤。否则，请直接快进到下一步：使用 Trainer。

### 定义模型的特征

`models/features.py` 包含用于定义模型特征的常量，包括特征名称、词汇表大小等。默认情况下，`penguin` 模板有两个常量（`FEATURE_KEYS` 和 `LABEL_KEY`），因为我们的 `penguin` 模型会使用监督学习解决分类问题，并且所有特征均为连续的数字特征。请参阅[芝加哥出租车示例中的特征定义](https://github.com/tensorflow/tfx/blob/master/tfx/experimental/templates/taxi/models/features.py)来查看另一个示例。


### 在 preprocessing_fn() 中实现针对训练/应用的预处理

实际的特征工程发生于 `models/preprocessing.py` 内的 `preprocessing_fn()` 函数中。

在 `preprocessing_fn` 中，您可以定义一系列函数来操作张量的输入字典以生成张量的输出字典。TensorFlow Transform API 中包含诸如 `scale_to_0_1` 和 `compute_and_apply_vocabulary` 等辅助函数，或者您也可以仅仅使用常规 TensorFlow 函数。默认情况下，`penguin` 模板包含 [tft.scale_to_z_score](https://tensorflow.google.cn/tfx/transform/api_docs/python/tft/scale_to_z_score) 函数归一化特征值的示例用法。

有关创作 `preprocessing_fn` 的详细信息，请参阅 [Tensflow Transform 转换指南](https://tensorflow.google.cn/tfx/transform/get_started)。


### 将 Transform 组件添加到流水线

如果您的 preprocessing_fn 已准备就绪，请将 `Transform` 组件添加到流水线。

1. 在 `pipeline/pipeline.py` 文件中，取消注释 `# components.append(transform)` 以将组件添加到流水线。

您可以更新流水线并再次运行。

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

如果流水线成功运行，您应在日志的*某处*看到“Component Transform is finished.”。由于 `Transform` 组件和 `ExampleValidator` 组件并不相互依赖，执行的顺序不是固定的。也就是说，`Transform` 和 `ExampleValidator` 中的任何一项都可以是流水线执行中的最后一个组件。

### 检查 Transform 的输出

Transform 组件会创建两种输出：Tensorflow 计算图和转换后的样本。转换后的样本为 Examples 工件类型，ExampleGen 也会生成这种类型，但转换后的样本包含转换后的特征值。

您可以采用我们在上一步中所用的方法进行检查。

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous run of Transform component.
    artifacts = get_latest_artifacts(metadata_handler.store,
                                     PIPELINE_NAME, "Transform")
    # Find artifacts of Examples type.
    transformed_examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)

In [ ]:
preview_examples(transformed_examples_artifacts)

## 第 4 步：使用 Trainer 组件训练模型

我们将使用 `Trainer` 组件构建机器学习模型。有关详情，请参阅 [Trainer 组件指南](https://tensorflow.google.cn/tfx/guide/trainer)。您需要对 Trainer 组件提供模型代码。

### 定义模型

在 penguin 模板中，`models.model.run_fn` 用作 `Trainer` 组件的 `run_fn` 参数。这意味着 `Trainer` 组件运行时会调用 `models/model.py` 中的 `run_fn()` 函数。您可以在给定的代码中看到使用 `keras` API 构建简单 DNN 模型的代码。有关在 TFX 中使用 keras API 的详细信息，请参阅 [TFX 中的 TensorFlow 2.x](https://tensorflow.google.cn/tfx/guide/keras) 指南。

在此 `run_fn` 中，您应构建一个模型并将其保存到由组件指定的 `fn_args.serving_model_dir` 指向的目录中。您可以在传递给 `run_fn` 的 `fn_args` 中使用其他参数。有关 `fn_args` 中参数的完整列表，请参阅[相关代码](https://github.com/tensorflow/tfx/blob/b01482442891a49a1487c67047e85ab971717b75/tfx/components/trainer/executor.py#L141)。

在 `models/features.py` 中定义您的特征并根据需要予以使用。如果您在第 3 步中对特征进行了转换，则应使用转换后的特征作为模型的输入。

### 将 Trainer 组件添加到流水线

如果您的 run_fn 已准备就绪，请将 `Trainer` 组件添加到流水线。

1. 在 `pipeline/pipeline.py` 文件中，取消注释 `# components.append(trainer)` 以将组件添加到流水线。

Trainer 组件的参数可能取决于您是否使用 Transform 组件。

- 如果您**不**使用 `Transform` 组件，则无需更改参数。

- 如果您使用 `Transform` 组件，则在创建 `Trainer` 组件实例时需要更改参数。

    - 将 `examples` 参数更改为 `examples=transform.outputs['transformed_examples'],`。我们需要使用转换后的样本进行训练。
    - 添加 `transform_graph` 参数，如 `transform_graph=transform.outputs['transform_graph'],`。此计算图包含用于转换运算的 TensorFlow 计算图。
    - 完成上述更改后，创建 Trainer 组件的代码将如下所示。

    ```python
    # If you use a Transform component.
    trainer = Trainer(
        run_fn=run_fn,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        ...
    ```

您可以更新流水线并再次运行。

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

在此执行成功运行后，您就已经为模型创建并运行了第一个 TFX 流水线。恭喜！

您的新模型将位于输出目录下的某个位置，但最好将模型放置在位于 TFX 流水线（其中会保存许多临时结果）之外的固定位置或服务当中。此外，也最好对机器学习生产系统中至关重要的构建模型进行持续评估。我们将在下一步中了解如何在 TFX 中进行持续评估和部署。

## 第 5 步（可选）：使用 Evaluator 评估模型并使用 Pusher 发布


[`Evaluator`](https://tensorflow.google.cn/tfx/guide/evaluator) 组件可持续评估 `Trainer` 构建的每个模型，[`Pusher`](https://tensorflow.google.cn/tfx/guide/pusher) 可将模型复制到文件系统中的预定义位置，甚至复制到 [Google Cloud AI Platform 模型](https://console.cloud.google.com/ai-platform/models)中。

### 将 Evaluator 组件添加到流水线

在 `pipeline/pipeline.py` 文件中：

1. 取消注释 `# components.append(model_resolver)` 以将最近的模型解析器添加到流水线。Evaluator 可用于将模型与在上一次流水线运行中通过 Evaluator 的旧基线模型进行比较。`LatestBlessedModelResolver` 可以查找最近通过 Evaluator 的模型。
2. 为您的模型设置适当的 `tfma.MetricsSpec`。每个机器学习模型的评估可能会有所不同。在 penguin 模板中使用了 `SparseCategoricalAccuracy`，因为我们解决的是多类别分类问题。您还需要指定 `tfma.SliceSpec` 来分析特定切片的模型。有关更多详情，请参阅 [Evaluator 组件指南](https://tensorflow.google.cn/tfx/guide/evaluator)。
3. 取消注释 `# components.append(evaluator)` 以将组件添加到流水线。

您可以更新流水线并再次运行。

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

### 检查 Evaluator 的输出

此步骤需要 TensorFlow Model Analysis (TFMA) Jupyter Notebook 扩展程序。请注意，TFMA Notebook 扩展程序的版本应与 TFMA Python 软件包的版本相同。

以下命令将从 NPM 注册表安装 TFMA Notebook 扩展程序。可能需要几分钟时间才能完成安装。

In [ ]:
# Install TFMA notebook extension.
!jupyter labextension install tensorflow_model_analysis@{tfma.__version__}

安装完成后，请**重新加载浏览器**以使扩展程序生效。

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
  # Search all aritfacts from the previous pipeline run.
  artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
  model_evaluation_artifacts = find_latest_artifacts_by_type(
      metadata_handler.store, artifacts,
      standard_artifacts.ModelEvaluation.TYPE_NAME)

In [ ]:
if model_evaluation_artifacts:
  tfma_result = tfma.load_eval_result(model_evaluation_artifacts[0].uri)
  tfma.view.render_slicing_metrics(tfma_result)

### 将 Pusher 组件添加到流水线

如果模型的准确率看起来不错，那么我们需要发布模型。[Pusher 组件](https://tensorflow.google.cn/tfx/guide/pusher)可以使用[自定义执行器](https://github.com/tensorflow/tfx/blob/master/tfx/extensions/google_cloud_ai_platform/pusher/executor.py)将模型发布到文件系统中的某个位置或 GCP AI Platform 模型中。

<a><code>Evaluator</code></a> 组件可以持续评估 <code>Trainer</code> 构建的每个模型，<a><code>Pusher</code></a> 可以将模型复制到文件系统中的预定义位置，甚至复制到 <a>Google Cloud AI Platform 模型</a>中。

1. 在 `local_runner.py` 中，将 `SERVING_MODEL_DIR` 设置为要发布到的目录。
2. 在 `pipeline/pipeline.py` 文件中，取消注释 `# components.append(pusher)` 以将组件添加到流水线。

您可以更新流水线并再次运行。

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

您应当可以在 `SERVING_MODEL_DIR` 找到新模型。

## 第 6 步（可选）：将流水线部署到 GCP 上的 Kubeflow Pipelines


如前所述，`local_runner.py` 非常适于调试或开发目的，但并非生产工作负载下的最佳解决方案。在此步骤中，我们会将流水线部署至 Google Cloud 上的 Kubeflow Pipelines。

### 准备

我们需要 `kfp` Python 软件包和 `skaffold` 程序来将流水线部署到 Kubeflow Pipelines 集群。

In [ ]:
!pip install --upgrade -q kfp

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold

您需要将 `skaffold` 二进制文件移动到您的 shell 可以找到的位置。或者，您也可以在运行 `tfx` 二进制文件时使用 `--skaffold-cmd` 标记指定 skaffold 的路径。

In [ ]:
# Move skaffold binary into your path
!mv skaffold /home/jupyter/.local/bin/

您还需要 Kubeflow Pipelines 集群来运行流水线。请按照[在 Cloud AI Platform Pipelines 上使用 TFX ](https://tensorflow.google.cn/tfx/tutorials/tfx/cloud-ai-platform-pipelines)教程中的第 1 步和第 2 步进行操作。

当您的集群准备就绪时，通过在 [Google Cloud Console 的 `Pipelines` 页面](http://console.cloud.google.com/ai-platform/pipelines)中点击 *Open Pipelines Dashboard* 来打开流水线信息中心。此页面的网址是用于请求流水线运行的 `ENDPOINT`。端点值为网址的 https:// 之后到 googleusercontent.com（包含）的所有内容。将您的端点置于以下代码块中。


In [ ]:
ENDPOINT='' # Enter your ENDPOINT here.

要在 Kubeflow Pipelines 集群中运行我们的代码，我们需要将代码打包到容器镜像中。部署我们的流水线时会自动构建镜像，您只需为您的镜像设置名称和容器注册表。在我们的示例中，我们将使用 [Google Container Registry](https://cloud.google.com/container-registry)，并将其命名为 `tfx-pipeline`。

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]

# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

### 设置数据位置

您的数据应当可以从 Kubeflow Pipelines 集群访问。如果您在本地环境中使用过数据，则可能需要将其上传到诸如 Google Cloud Storage 的远程存储空间。例如，我们可以将企鹅数据上传到在部署 Kubeflow Pipelines 集群时自动创建的默认存储分区，如下所示。

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/penguin/

在 `kubeflow_runner.py` 中，更新存储在 `DATA_PATH` 的数据位置。

如果您使用 BigQueryExampleGen，则无需上传数据文件，但请确保 `kubeflow_runner.py` 对 `pipeline.create_pipeline()` 函数使用相同的 `query` 和 `beam_pipeline_args` 参数。

### 部署流水线

一切准备就绪后，您就可以使用 `tfx pipeline create` 命令创建流水线。

> 注：在为 Kubeflow Pipelines 创建流水线时，我们需要一个容器镜像，该镜像将用于运行流水线。`skaffold` 将为我们构建镜像。由于 `skaffold` 会从 Docker Hub 拉取基础镜像，在我们第一次构建镜像时，将花费 5~10 分钟，但从第二次构建开始，花费的时间会少得多。


In [ ]:
!tfx pipeline create  \
--engine=kubeflow \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

现在，使用 `tfx run create` 命令以新创建的流水线开始执行运行。

In [ ]:
!tfx run create --engine=kubeflow --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

或者，您也可以在 Kubeflow Pipelines 信息中心中运行流水线。新的运行将在 Kubeflow Pipelines 信息中心的 `Experiments` 下列出。点击进入实验后，您可以监视进度并呈现在执行运行期间创建的工件。

如果您有兴趣在 Kubeflow Pipelines 上运行您的流水线，请参阅[在 Cloud AI Platform Pipelines 上使用 TFX](https://tensorflow.google.cn/tfx/tutorials/tfx/cloud-ai-platform-pipelines) 教程以获取更多说明。

### 清理

要清理此步骤中使用的所有 Google Cloud 资源，您可以[删除用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以通过访问每个控制台来清理各个资源：

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)